# Parks and Recreation Data Cleaning and Analysis

Load raw data file `City_Facilities_-_Recreation_and_Parks_Jurisdiction_or_Leased_20251109.csv` 
from the raw data folder. Use necessary libraries to for data cleaning, analysis, and visualization.

The raw data for this was downloaded from [DataSF | Open Data Portal - City Facilities - Recreation and Parks Jurisdiction or Leased](https://data.sfgov.org/City-Infrastructure/City-Facilities-Recreation-and-Parks-Jurisdiction-/2mqz-ak5w/about_data) 

In [63]:
# import necessary dependencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from census import Census
from us import states
import os
from pathlib import Path
from IPython.display import display
import seaborn as sns
import re


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# use robust path handling to locate the data file
def find_repo_root(start: Path = Path.cwd()) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'requirements.txt').exists() or (p / '.git').exists():
            return p
    return start

repo_root = find_repo_root()
data_file = repo_root / 'data' / 'raw' / 'City_Facilities_-_Recreation_and_Parks_Jurisdiction_or_Leased_20251109.csv'

if not data_file.exists():
    raise FileNotFoundError(f"Data file not found at {data_file}. Check the file name and data/raw folder.")

# load the data from the data/raw folder
parks_and_rec_df = pd.read_csv(data_file, low_memory=False)

### Parks and Recreation City Facilities EDA

#### Initial Look at the Raw Data

In [ ]:
# see the first 5 rows of the data
display(parks_and_rec_df.head())

,facility_id,common_name,address,city,zip_code,block_lot,owned_leased,dept_id_for_jurisdiction,jurisdiction,gross_sq_ft,longitude,latitude,supervisor_district,city_tenants
0,3520,240 6th St,240 6th St,San Francisco,94103,3731004,Own,49,Recreation And Parks,NaN,-122.406798,37.778960,6.0,NaN
1,3342,500 Pine Rooftop Park,500 Pine St,San Francisco,94108,0258042,Own,49,Recreation And Parks,NaN,-122.404581,37.791879,3.0,NaN
2,3339,900 Innes Ave,900 Innes Ave,San Francisco,94124,4629A010,Own,49,Recreation And Parks,NaN,-122.375781,37.732187,10.0,NaN
3,10,Adam Rogers Park Bathrooms,45 George Ct,San Francisco,94124,4700055,Own,49,Recreation And Parks,NaN,-122.383705,37.730880,10.0,NaN
4,108,Alamo Square Bathrooms,1081 Fulton St,San Francisco,94117,0799001,Own,49,Recreation And Parks,500,-122.434338,37.776070,5.0,NaN


In [ ]:
# seeing the number of rows and columns in the data
print("The number of (rows, columns) in this DataFrame is", parks_and_rec_df.shape)

# seeing the column names in the data
print("The columns in this DataFrame are:\n", parks_and_rec_df.columns.tolist())

The number of (rows, columns) in this DataFrame is (574, 14)
The columns in this DataFrame are:
 ['facility_id', 'common_name', 'address', 'city', 'zip_code', 'block_lot', 'owned_leased', 'dept_id_for_jurisdiction', 'jurisdiction', 'gross_sq_ft', 'longitude', 'latitude', 'supervisor_district', 'city_tenants']


In [ ]:
# getting summary information about the data including # null, data types etc.
parks_and_rec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   facility_id               574 non-null    int64  
 1   common_name               574 non-null    object 
 2   address                   574 non-null    object 
 3   city                      574 non-null    object 
 4   zip_code                  574 non-null    int64  
 5   block_lot                 572 non-null    object 
 6   owned_leased              574 non-null    object 
 7   dept_id_for_jurisdiction  574 non-null    int64  
 8   jurisdiction              574 non-null    object 
 9   gross_sq_ft               235 non-null    object 
 10  longitude                 574 non-null    float64
 11  latitude                  574 non-null    float64
 12  supervisor_district       458 non-null    float64
 13  city_tenants              4 non-null      object 
dtypes: float64

In [ ]:
# getting summary statistics for numerical columns
parks_and_rec_df.describe()

,facility_id,zip_code,dept_id_for_jurisdiction,longitude,latitude,supervisor_district
count,574.000000,574.000000,574.000000,574.000000,574.000000,458.000000
mean,1198.630662,94353.594077,48.721254,-121.948965,37.783527,5.515284
std,920.896395,476.807582,3.330383,1.032795,0.053428,3.012154
min,1.000000,94044.000000,9.000000,-122.510206,37.624935,1.000000
25%,456.500000,94117.000000,49.000000,-122.477980,37.747821,3.000000
50%,988.000000,94118.000000,49.000000,-122.447323,37.769459,7.000000
75%,1632.750000,94133.000000,49.000000,-122.405449,37.795794,7.000000
max,3520.000000,95321.000000,49.000000,-119.851863,37.883704,11.000000


Based on this initial glance of the data, this are my main insights and next steps for data cleaning:
- Some of the datatypes are mislabeled. For example, `facility_id`, `zip_code`, `dept_id_for_jurisdiction`, and `supervisor_district` should be an object. On the other hand, `gross_sq_ft` should be an numeric (like float) data point.
- Less than half of the columns have non-missing data for `gross_sq_ft` 235/574 - to be exact, so we need to figure out how we can handle this missing data as this value is important for our analysis. Looking at the Open Data Portal page for this dataset, they have noted that the original source of this data is FRRM and that this is "incomplete data, updating as values are determined." This value is important for us because we are looking at the per-capita availability of parks and recreation facilities at the tract level.
- This dataset maps Parks and Recreation facilities to the `block_lot` level when we are interested in the Census Tract level. We need to figure out how to convert the data to that level.

#### Data Cleaning

As listed above, I started off converting mislabeled datatypes to their respective correct types.

In [ ]:
# fixing mislabeled datatypes
parks_and_rec_df['facility_id'] = parks_and_rec_df['facility_id'].astype(str)
parks_and_rec_df['zip_code'] = parks_and_rec_df['zip_code'].astype(str)
parks_and_rec_df['dept_id_for_jurisdiction'] = parks_and_rec_df['dept_id_for_jurisdiction'].astype(str)
parks_and_rec_df['supervisor_district'] = parks_and_rec_df['supervisor_district'].astype(str)

# holding off on converting gross_sq_ft to numeric due to many missing values --> want to explore more before converting
#parks_and_rec_df['gross_sq_ft'] = pd.to_numeric(parks_and_rec_df['gross_sq_ft'], errors='coerce')  

# checking the datatypes after fixing them
parks_and_rec_df.dtypes

facility_id                  object
common_name                  object
address                      object
city                         object
zip_code                     object
block_lot                    object
owned_leased                 object
dept_id_for_jurisdiction     object
jurisdiction                 object
gross_sq_ft                  object
longitude                   float64
latitude                    float64
supervisor_district          object
city_tenants                 object
dtype: object

Now, I'll look deeper into `city` and `zip_code` columns to identify if there are any redundant columns. In this case, there aren't any. There are a few parks and zip codes in neighboring cities next to SF like Pacifica (which makes sense) parks can extend SF boundary. However, this also doesn't make sense because Groveland (for example) is not in SF or neighboring at all. I'll uncover something about this below when I look further at the missing data in the `gross_sq_ft` column in a bit.

In [ ]:
# understanding the unique values in 'city' column
display(parks_and_rec_df["city"].value_counts())

# understand the unique values in 'zip_code' column
display(parks_and_rec_df["zip_code"].value_counts())

city
San Francisco    460
Groveland        112
Pacifica           2
Name: count, dtype: int64

zip_code
94117    158
95321    112
94132     50
94121     25
94134     23
94110     20
94112     18
94116     18
94123     17
94124     15
94122     14
94118     13
94108     12
94115     11
94133     10
94102      9
94131      9
94109      8
94114      7
94107      7
94127      6
94103      5
94111      4
94044      2
94105      1
Name: count, dtype: int64

Moving onto looking at the `gross_sq_ft` column to conver the existing values to numeric data types and make an educated decision to handle the missing data.

In [ ]:
# exploring the gross_sq_ft column to see what the values look like
display(parks_and_rec_df['gross_sq_ft'].value_counts())
display(parks_and_rec_df['gross_sq_ft'].describe())
display(parks_and_rec_df['gross_sq_ft'].head(10))

gross_sq_ft
1200.0     15
500.0       8
2000.0      5
1500.0      4
432.0       3
           ..
8001.0      1
576.0       1
19446.0     1
2187.0      1
2135.0      1
Name: count, Length: 185, dtype: int64

count       235.000000
mean       6234.608511
std       20742.788663
min          38.000000
25%         679.500000
50%        1578.000000
75%        4872.000000
max      300000.000000
Name: gross_sq_ft, dtype: float64

0       NaN
1       NaN
2       NaN
3       NaN
4     500.0
5    2505.0
6       NaN
7     432.0
8    1200.0
9    2196.0
Name: gross_sq_ft, dtype: float64

What I notice about the `gross_sq_ft` column is that there are commas in the numbers (ex. 1,200), so I need to take care for that when converting to numeric data types. Also, out of the 235 non-null values, 185 are unique, which suggests that some of the values are estimates of the gross sq. ft. of these facilities. This could be useful for handling the missing data.

I'll convert the existing non-null values of `gross_sq_ft` column correctly below:

In [ ]:
# convert gross_sq_ft to numeric safely, keeping original values for reference
parks_and_rec_df['gross_sq_ft_raw'] = parks_and_rec_df['gross_sq_ft'].astype(object)

# remove commas and any non-digit/decimal/minus characters, convert empty -> NaN
parks_and_rec_df['gross_sq_ft_clean'] = (
    parks_and_rec_df['gross_sq_ft_raw']
    .astype(str)
    .str.replace(r'[^\d\.\-]', '', regex=True)   # strips commas and non-numeric characters
    .replace('', np.nan)
)

# final numeric conversion (coerce invalid -> NaN)
parks_and_rec_df['gross_sq_ft'] = pd.to_numeric(parks_and_rec_df['gross_sq_ft_clean'], errors='coerce')
parks_and_rec_df.drop(columns=['gross_sq_ft_clean'], inplace=True)  # drop clean column to avoid confusion

# checking that there are 235 non-null values in gross_sq_ft after conversion
print("numeric non-null:", parks_and_rec_df['gross_sq_ft'].notna().sum())

numeric non-null: 235


In [ ]:
# looking at the rows where gross_sq_ft is still NaN after conversion to see if there are any patterns
display(parks_and_rec_df[parks_and_rec_df['gross_sq_ft'].isna()].groupby('jurisdiction').agg({'facility_id':'count'}).sort_values(by='facility_id', ascending=False))
display(parks_and_rec_df[parks_and_rec_df['gross_sq_ft'].isna()].groupby('city').agg({'facility_id':'count'}).sort_values(by='facility_id', ascending=False))
display(parks_and_rec_df[parks_and_rec_df['gross_sq_ft'].isna()].groupby('supervisor_district').agg({'facility_id':'count'}).sort_values(by='facility_id', ascending=False))
display(parks_and_rec_df[parks_and_rec_df['gross_sq_ft'].isna()].groupby('address').agg({'facility_id':'count'}).sort_values(by='facility_id', ascending=False))

,facility_id
jurisdiction,
Recreation And Parks,335
Non-City,4


,facility_id
city,
San Francisco,225
Groveland,112
Pacifica,2


,facility_id
supervisor_district,
nan,114
7.0,80
4.0,30
1.0,27
10.0,16
3.0,14
9.0,14
5.0,13
2.0,10


,facility_id
address,
32560 Mather Rd,110
1 Zoo Rd,31
100 Martin Luther King Dr,20
100 John F Kennedy Dr,7
1199 09th Ave,4
...,...
2301 San Jose Ave,1
2100 Martin Luther King Jr Dr,1
21 CHAIN OF LAKES DR,1


In [66]:
# looking deeper at the "1 Zoo Rd" address
parks_and_rec_df[parks_and_rec_df['address'] == "1 Zoo Rd"].head(5)

,facility_id,common_name,address,city,zip_code,block_lot,owned_leased,dept_id_for_jurisdiction,jurisdiction,gross_sq_ft,longitude,latitude,supervisor_district,city_tenants,gross_sq_ft_raw
543,3126,ZOO - ANIMAL HOSPITAL*,1 Zoo Rd,San Francisco,94132,7281002,Own,49,Recreation And Parks,NaN,-122.503913,37.731741,7.0,NaN,NaN
544,3127,ZOO - AVIARY*,1 Zoo Rd,San Francisco,94132,7281002,Own,49,Recreation And Parks,NaN,-122.503223,37.732559,7.0,NaN,NaN
545,3128,ZOO - BEAR DENS*,1 Zoo Rd,San Francisco,94132,7281002,Own,49,Recreation And Parks,NaN,-122.501909,37.731501,7.0,NaN,NaN
546,3129,ZOO - CAPYBARA SHELTER*,1 Zoo Rd,San Francisco,94132,7281002,Own,49,Recreation And Parks,NaN,-122.501543,37.731567,7.0,NaN,NaN
547,3130,ZOO - CAROUSEL,1 Zoo Rd,San Francisco,94132,7281002,Own,49,Recreation And Parks,NaN,-122.503791,37.733596,7.0,NaN,NaN


In [67]:
# taking a deeper look at 100 Martin Luther King Dr	
parks_and_rec_df[parks_and_rec_df['address'] == "100 Martin Luther King Dr"]

,facility_id,common_name,address,city,zip_code,block_lot,owned_leased,dept_id_for_jurisdiction,jurisdiction,gross_sq_ft,longitude,latitude,supervisor_district,city_tenants,gross_sq_ft_raw
20,146,Auto Shop,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,6050.0,-122.461078,37.768702,7.0,NaN,6050.0
174,537,Corporate Yard Maintenance Facility,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.460935,37.768669,7.0,NaN,NaN
175,538,Corporate Yard Storage,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.461108,37.768505,7.0,NaN,NaN
176,533,Corp Yard Gas Station,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,5929.0,-122.460616,37.768949,7.0,NaN,5929.0
178,3051,CORP. YARD P.PATROL/SHOPS*,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.461203,37.767393,7.0,NaN,NaN
179,3050,CORP YARD / Warehouse,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.461203,37.767393,7.0,NaN,NaN
187,589,Double Dutch Head House,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.509173,37.770449,1.0,NaN,NaN
195,630,Electrical & Iron Shops,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,4488.0,-122.461982,37.768429,7.0,NaN,4488.0
197,634,Emergency Equipment Storage,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.461564,37.768373,7.0,NaN,NaN
221,801,GGP Greenhouses,100 Martin Luther King Dr,San Francisco,94117,1263011,Own,49,Recreation And Parks,NaN,-122.461843,37.768708,7.0,NaN,NaN


After exploring how the `gross_sq_ft` column relates to some of the other variables above, here are some things that stand out to me:
- All the Groveland data has missing gross_sq_feet. This is important because Groveland is not neighboring SF (at all). The most common address in this category 32560 Mather Rd is a part of Stanislaus National Park/Yosemite National Park, which I found out by looking at this location in Google Maps. This isn't an SF facility, but is including in the dataset because a significant portion of the SFPUC's water system facilities are located in Tuolumne County. These values can be dropped as they don't affect our research in terms of parks in SF.
- The next common address 1 Zoo Rd, corresponds to the SF Zoo and specific attraction in the zoo like Aviary or Bear Dens. The reason the gross_sq_ft is not updated for each of these values is likely because the area of the inner attractions keep changing. Instead, I can handle this data by taking the area (100 acres ~ 4.356 million sq ft) of the San Franciso Zoo, and replacing these values with just 1 value corresponding to the total area. One thing to note about this method of analysis is that the census tract that the SF Zoo is in 06075060400  mainly includes just the zoo and Lake Merced Park, so basically the whole census tract can be classified under "Parks and Recreation" area.